In [ ]:
import cv2
import pickle
from utils import show
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

In [7]:
# 1️⃣ Cargar parámetros de calibración estéreo
stereo_params_path = "stereo_params.pkl"
with open(stereo_params_path, 'rb') as f:
    stereo_data = pickle.load(f)

K1 = stereo_data["K1"]
dist1 = stereo_data["dist1"]
K2 = stereo_data["K2"]
dist2 = stereo_data["dist2"]
R = stereo_data["R"]
T = stereo_data["T"]
P1 = stereo_data["P1"]
P2 = stereo_data["P2"]
map1x = stereo_data["map1x"]
map1y = stereo_data["map1y"]
map2x = stereo_data["map2x"]
map2y = stereo_data["map2y"]
image_size = stereo_data["image_size"]

In [8]:
# 2️⃣ Cargar primer par de imágenes estéreo
left_img_path = "datasets/captures/left_0.jpg"
right_img_path = "datasets/captures/right_0.jpg"

left_img_raw = cv2.imread(left_img_path, cv2.IMREAD_GRAYSCALE)
right_img_raw = cv2.imread(right_img_path, cv2.IMREAD_GRAYSCALE)

In [9]:
# 3️⃣ Rectificación del primer frame
left_img = cv2.remap(left_img_raw, map1x, map1y, cv2.INTER_LINEAR)
right_img = cv2.remap(right_img_raw, map2x, map2y, cv2.INTER_LINEAR)

show(left_img, "Left rectified")
show(right_img, "Right rectified")

NameError: name 'show' is not defined

In [ ]:
# 4️⃣ Extracción y matching de features entre img izq y der
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(left_img, None)
kp2, des2 = sift.detectAndCompute(right_img, None)

# BFMatcher con L2 para SIFT
top_matches = 2000
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = matcher.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)[:top_matches]

# Visualizar matches
img_matches = cv2.drawMatches(left_img, kp1, right_img, kp2, matches, None, flags=2)
show(img_matches, "Stereo matches (left ↔ right)")

In [ ]:
# 5️⃣ Triangulación inicial de puntos 3D
# Preparamos puntos en formato Nx2
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

# Convertir a coordenadas homogéneas para triangulación
pts1_h = cv2.undistortPoints(np.expand_dims(pts1, axis=1), K1, dist1, P=P1)
pts2_h = cv2.undistortPoints(np.expand_dims(pts2, axis=1), K2, dist2, P=P2)

# Triangular
points_4d_hom = cv2.triangulatePoints(P1, P2, pts1_h, pts2_h)
points_3d = (points_4d_hom / points_4d_hom[3])[:3].T

print("🔹 Puntos 3D triangulados:", points_3d.shape)

# Visualización simple de la nube de puntos (XY)
plt.scatter(points_3d[:, 0], points_3d[:, 1], s=1)
plt.title("Proyección XY de la nube de puntos inicial")
plt.axis("equal")
plt.show()